In [4]:
import json

import os

import requests

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

SONO_API_URL = os.getenv("SONO_API_URL")
GPT_API_URL = os.getenv("GPT_API_URL")
GPT_API_KEY = os.getenv("GPT_API_KEY")

def generate_song(text_prompt):

    ROLE_intro = "You are an amazing songwriter assistant and DJ. Giving you some of the following fields [Title, Language, Generes, Idea] you must fill in the missing ones AND provide lyrics with the right language (preferably english, italian or spanish) and coherent with the concpets described by the Idea and the Title (if provided)." 
    lyrics = " Divide the lyrics in sections and specify the section type with tags among these: [Intro], [Outro], [Chorus], [Bridge], [Verse], [Pre-Chorus], [Instrumental], [Drop]. Don't get above 1000 characters (counting tags)"

    genres = " If the Genres are missing, they must be a list of up to 4 musical genres that can represent the mood of the Idea."
    genre_bias = " Please have a small bias toward \"dancy\"/not \"boring\" genres as you will be deployed to get some background music during an event."

    ans_format = " Answare with the format:\n\nTitle: $Title\nLanguage: $Language\nGenres: $Genre1, $Genre2, ...\n$Idea: $Idea\n\nLyrics:\n$Lyrics."

    ROLE = ROLE_intro + lyrics + genres+ genre_bias + ans_format


    # Step 1: Generate text using GPT

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {GPT_API_KEY}',
    }

    # gpt_payload = {
    #     "model": "gpt-3.5-turbo",
    #     'messages': [{'role': 'user', 'content': text_prompt}],
    #     'functions': [{
    #         'name': 'generate_song',
    #         'description': ROLE,
    #         'parameters': {
    #             'type': 'object',
    #             'properties': {
    #                 'title': {'type': 'string'},
    #                 'language': {'type': 'string'},
    #                 'genres': {'type': 'string'},
    #                 'idea': {'type': 'string'},  
    #                 'lyrics': {'type': 'string'},
    #             },
    #             'required': ['prompt', 'title'],
    #     },}],
    #     'function_call': {'name': 'generate_song'},
    # }

    gpt_payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": ROLE
            },
            {
                "role": "user",
                "content": text_prompt
            }
        ]
    }

    gpt_response = requests.post(GPT_API_URL, headers=headers, json=gpt_payload)
    gpt_response.raise_for_status()  # Raise an error for bad status codes

    #gpt_data = gpt_response.json()
    # function_call_result = (
    #     gpt_data.get('choices', [{}])[0]
    #     .get('message', {})
    #     .get('function_call', {})
    #     .get('arguments', '{}')
    # )
    # function_call_result = json.loads(function_call_result)
    # generated_prompt = function_call_result.get('prompt')
    # generated_title = function_call_result.get('title')
    # generated_tags = function_call_result.get('genres')

    return gpt_response.json()['choices'][0]['message']['content'] #function_call_result #generated_prompt, generated_title, generated_tags

In [6]:
res = generate_song("Idea: una canzone sulla panda, autoveicolo italico. Titolo: La danza della Panda")

In [7]:
print(res)

Title: La danza della Panda
Language: Italian
Genres: Pop, Dance, Electronic
Idea: A lively song celebrating the Italian car "Panda" with catchy and upbeat lyrics.

Lyrics:
[Verse]
Nella strada la Panda va,
Piccola ma veloce, sfreccia già.
Con il motore che ronza forte,
La danza della Panda ci riporta all'istante.

[Pre-Chorus]
Con le sue curve aggraziate,
La Panda ci fa sognare.
Insieme a lei, niente può fermarci,
La felicità su quattro ruote, è lei che ce la dà.

[Chorus]
La danza della Panda, brillante e leggera,
Ci porta lontano, ci fa sentire in primavera.
Sulla strada, in libertà, la Panda ci fa ballare,
Insieme a lei, niente può fermarci, solo sognare.

[Verse]
In città o in campagna, lei non teme la fatica,
La Panda è sempre pronta, una fedele amica.
Con i suoi colori alla moda,
La danza della Panda è la più ambita.

[Bridge]
E quando il sole tramonta lentamente,
Nella Panda siamo ancora più contenti.
Guardando le stelle accendersi in cielo,
La danza della Panda è il nostro più

In [8]:
import requests

url = SONO_API_URL+'/api/custom_generate'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
data = {
    "prompt": "[Verse 1]\nCruel flames of war engulf this land\nBattlefields filled with death and dread\nInnocent souls in darkness, they rest\nMy heart trembles in this silent test\n\n[Verse 2]\nPeople weep for loved ones lost\nBattered bodies bear the cost\nSeeking peace and hope once known\nOur grief transforms to hearts of stone\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Bridge]\nThrough the ashes, we will rise\nHand in hand, towards peaceful skies\nNo more sorrow, no more pain\nTogether, we'll break these chains\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Outro]\nIn unity, our strength will grow\nA brighter future, we'll soon know\nFrom the ruins, hope will spring\nA new dawn, we'll together bring",
    "tags": "pop metal male melancholic",
    "title": "Silent Battlefield",
    "make_instrumental": False,
    "wait_audio": False
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Error:", response.status_code, response.text)


Success: [{'id': '51c5b46c-c05a-4ebe-b921-589072c8b011', 'title': 'Silent Battlefield', 'image_url': None, 'lyric': "[Verse 1]\nCruel flames of war engulf this land\nBattlefields filled with death and dread\nInnocent souls in darkness, they rest\nMy heart trembles in this silent test\n\n[Verse 2]\nPeople weep for loved ones lost\nBattered bodies bear the cost\nSeeking peace and hope once known\nOur grief transforms to hearts of stone\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Bridge]\nThrough the ashes, we will rise\nHand in hand, towards peaceful skies\nNo more sorrow, no more pain\nTogether, we'll break these chains\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Outro]\nIn unity, our strength will grow\nA brighter future

In [10]:
response.json()

[{'id': '51c5b46c-c05a-4ebe-b921-589072c8b011',
  'title': 'Silent Battlefield',
  'image_url': None,
  'lyric': "[Verse 1]\nCruel flames of war engulf this land\nBattlefields filled with death and dread\nInnocent souls in darkness, they rest\nMy heart trembles in this silent test\n\n[Verse 2]\nPeople weep for loved ones lost\nBattered bodies bear the cost\nSeeking peace and hope once known\nOur grief transforms to hearts of stone\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Bridge]\nThrough the ashes, we will rise\nHand in hand, towards peaceful skies\nNo more sorrow, no more pain\nTogether, we'll break these chains\n\n[Chorus]\nSilent battlegrounds, no birds' song\nShadows of war, where we don't belong\nMay flowers of peace bloom in this place\nLet's guard this precious dream with grace\n\n[Outro]\nIn unity, our strength will grow\nA brighter future, w